# Assignment 1: Time Series Forecast With Python (Seasonal ARIMA)

**Lecturer**: Vincent Claes<br>
**Authors:** Bryan Honof, Jeffrey Gorissen<br>
**Start Date:** 19/10/2018
    
**Objective:** Visualize and predict the future temperatures via ARIMA

**Description:** In this notebook we train our model

In [1]:
import math
import warnings
import datetime

import pandas            as pd
import itertools         as it
import statsmodels.api   as sm
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore") # specify to ignore warning messages

In [2]:
data_csv = pd.read_csv('rolmean_data.csv')
data = pd.DataFrame()

# Convert the creation_date column to datetime64
data['dateTime'] = pd.to_datetime(data_csv['dateTime'])
# Convert the value column to float
data['temperature'] = pd.to_numeric(data_csv['temperature'])

# Set the dateTime column as index
data = data.set_index(['dateTime'])

# Sort the dataFrame just to be sure...
data = data.sort_index()

data = data.dropna()

# Double check the results
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 824 entries, 2018-11-11 03:00:00 to 2018-11-19 16:45:00
Data columns (total 1 columns):
temperature    824 non-null float64
dtypes: float64(1)
memory usage: 12.9 KB


In [3]:
data.tail(5)

,temperature
dateTime,
2018-11-19 15:45:00,17.07
2018-11-19 16:00:00,17.28
2018-11-19 16:15:00,17.27
2018-11-19 16:30:00,17.28
2018-11-19 16:45:00,17.28


In [4]:
# Tune Seasonal ARIMA model
# Define the p, d and q parameters to take any value between 0 and 2
p = d = q = range(0,2)

# Generate all different combinations of p, q and q triplets
pdq = list(it.product(p, d, q))
print(pdq)

# Generate all different combinations of seasonal p, q and q triplets
# Seasonality is one week (24*7 = 168 hours)
seasonal_pdq = [(x[0], x[1], x[2], 15) for x in list(it.product(p, d, q))]

print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

[(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)]
Examples of parameter combinations for Seasonal ARIMA...
SARIMAX: (0, 0, 1) x (0, 0, 1, 15)
SARIMAX: (0, 0, 1) x (0, 1, 0, 15)
SARIMAX: (0, 1, 0) x (0, 1, 1, 15)
SARIMAX: (0, 1, 0) x (1, 0, 0, 15)


In [5]:
train = data

train.index = pd.DatetimeIndex(train.index.values)

result_list = []

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(data,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)
            results = mod.fit()

            print('ARIMA{}x{} - AIC:{}'.format(param, param_seasonal, round(results.aic,2)))
            result_list.extend([param, param_seasonal, round(results.aic,2)])
        except:
            print('error')
            continue
            
print('Done!')

ARIMA(0, 0, 0)x(0, 0, 0, 15) - AIC:7121.11
ARIMA(0, 0, 0)x(0, 0, 1, 15) - AIC:5959.59
ARIMA(0, 0, 0)x(0, 1, 0, 15) - AIC:2503.9
ARIMA(0, 0, 0)x(0, 1, 1, 15) - AIC:2464.15
ARIMA(0, 0, 0)x(1, 0, 0, 15) - AIC:2504.66
ARIMA(0, 0, 0)x(1, 0, 1, 15) - AIC:2496.19
ARIMA(0, 0, 0)x(1, 1, 0, 15) - AIC:2470.75
ARIMA(0, 0, 0)x(1, 1, 1, 15) - AIC:2456.74
ARIMA(0, 0, 1)x(0, 0, 0, 15) - AIC:5987.85
ARIMA(0, 0, 1)x(0, 0, 1, 15) - AIC:4919.78
ARIMA(0, 0, 1)x(0, 1, 0, 15) - AIC:1525.14
ARIMA(0, 0, 1)x(0, 1, 1, 15) - AIC:1509.98
ARIMA(0, 0, 1)x(1, 0, 0, 15) - AIC:1527.26
ARIMA(0, 0, 1)x(1, 0, 1, 15) - AIC:1529.62
ARIMA(0, 0, 1)x(1, 1, 0, 15) - AIC:1512.64
ARIMA(0, 0, 1)x(1, 1, 1, 15) - AIC:1508.66
ARIMA(0, 1, 0)x(0, 0, 0, 15) - AIC:-1231.37
ARIMA(0, 1, 0)x(0, 0, 1, 15) - AIC:-1339.32
ARIMA(0, 1, 0)x(0, 1, 0, 15) - AIC:-473.06
ARIMA(0, 1, 0)x(0, 1, 1, 15) - AIC:-1119.55
ARIMA(0, 1, 0)x(1, 0, 0, 15) - AIC:-1263.25
ARIMA(0, 1, 0)x(1, 0, 1, 15) - AIC:-1392.84
ARIMA(0, 1, 0)x(1, 1, 0, 15) - AIC:-756.37
ARIMA(0

In [7]:
print_result = zip(*[iter(result_list)]*3) 
print_result = sorted(print_result, key=lambda x: x[1])

print('Result summary:\n')
print('((p, d, q), (P, D, Q, S), AIC)')
print('------------------------------')
for item in print_result:
    print(item)

Result summary:

((p, d, q), (P, D, Q, S), AIC)
------------------------------
((0, 0, 0), (0, 0, 0, 15), 7121.11)
((0, 0, 1), (0, 0, 0, 15), 5987.85)
((0, 1, 0), (0, 0, 0, 15), -1231.37)
((0, 1, 1), (0, 0, 0, 15), -1400.3)
((1, 0, 0), (0, 0, 0, 15), -1231.87)
((1, 0, 1), (0, 0, 0, 15), -1401.43)
((1, 1, 0), (0, 0, 0, 15), -1529.67)
((1, 1, 1), (0, 0, 0, 15), -1673.66)
((0, 0, 0), (0, 0, 1, 15), 5959.59)
((0, 0, 1), (0, 0, 1, 15), 4919.78)
((0, 1, 0), (0, 0, 1, 15), -1339.32)
((0, 1, 1), (0, 0, 1, 15), -1619.46)
((1, 0, 0), (0, 0, 1, 15), -1205.78)
((1, 0, 1), (0, 0, 1, 15), -1365.05)
((1, 1, 0), (0, 0, 1, 15), -1902.36)
((1, 1, 1), (0, 0, 1, 15), -2111.68)
((0, 0, 0), (0, 1, 0, 15), 2503.9)
((0, 0, 1), (0, 1, 0, 15), 1525.14)
((0, 1, 0), (0, 1, 0, 15), -473.06)
((0, 1, 1), (0, 1, 0, 15), -583.0)
((1, 0, 0), (0, 1, 0, 15), -477.73)
((1, 0, 1), (0, 1, 0, 15), -590.25)
((1, 1, 0), (0, 1, 0, 15), -655.44)
((1, 1, 1), (0, 1, 0, 15), -764.67)
((0, 0, 0), (0, 1, 1, 15), 2464.15)
((0, 0, 1), 